# This notebook was created to demonstrate the operation of an adapter for CodeLlama, designed to solve the problem of improving the readability of decompiled C code

# 1) Installing the necessary libraries

In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install transformers

# 2) Import block

In [ ]:
from datetime import datetime
import os
import sys
import torch
from peft import get_peft_model, PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

# 3) Set device, basic model's tokenizer

In [ ]:
# Set which device we will use (GPU or CPU).
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# The basic model
BASE_MODEL = "codellama/CodeLlama-7b-hf"
# Initializing the tokenizer
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

# 4) Upload our eval dataset with code examples (optional)

In [ ]:
from datasets import load_dataset
eval_dataset = load_dataset('csv', data_files='/kaggle/input/95131hr/train_dataset_HR.csv', split='train[0%:1%]')
print(len(eval_dataset))

In [ ]:
# Example of decompiler output:
print(eval_dataset[1]['HR'])
test = eval_dataset[1]['HR']

# 5) Download the model together with DecLlama checkpoint

In [ ]:
#load the basic model
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
    )
)

In [ ]:
# load checkpoint
model = PeftModel.from_pretrained(model, '/kaggle/input/norm63000')
model.config.pad_token_id = tokenizer.pad_token_id = 0 
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [ ]:
# Architecture of the model with an adapter
print(model)

# 6) Evaluation

In [ ]:
#function for generating model prediction
def get_pred(dec, model, max_tokens):
    eval_prompt = f"""You are a powerful decompiler model. Your job is to convert the С code decompiled using RetDec decompiler into a more human-readable form. That is, you should change the names of variables and functions and delete unnecessary parts of the code so that it looks more like the source code of a C program. You are given a С code decompiled using RetDec decompiler. You must output the source code of a C program.

### Your input:
{dec}
### The original C program:
        """
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        pred = (tokenizer.decode(model.generate(**model_input, max_new_tokens=max_tokens)[0], skip_special_tokens=True))
    j = 0
    while (pred[j]+pred[j+1]+pred[j+2]+pred[j+3]+pred[j+4])!='### T':
        j+=1
    pred=pred[j::]
    pred = pred[39::]
    return (pred)

In [ ]:
pred = get_pred(test,model,1000)
print(pred)